In [1]:
import pandas as pd
#df = pd.read_parquet('sources/metadata.parquet', engine='pyarrow')
df_large = pd.read_parquet('sources/metadata-large.parquet', engine='pyarrow')

exact_matches = pd.read_parquet('results/artists_exact_match_large.parquet', engine='pyarrow')
assert "artists" in exact_matches.columns, f'artists is not in {exact_matches.columns}'
assert "num_artists" in exact_matches.columns, f'num_artists is not in {exact_matches.columns}'


# read excel_artist_names
import my_utils

excel_artist_names = my_utils.read_lines_as_list("sources/excel_artists_copy_paste_name.txt")
hundred_artist_names = excel_artist_names[0:100]
ten_artist_names = excel_artist_names[0:10]


artist_mentions = pd.read_parquet('results/artist_mentions.parquet', engine='pyarrow')

In [ ]:
df.shape

In [ ]:
len(df["prompt"].unique())

# Artist Mentions

In [ ]:

search_padding = 10

# we could try to check if some of the artists are more refered to by their last name
print(f'{artist_mentions.loc[ artist_mentions["mentions"] == 0 ].shape[0]} artists were not mentioned')
print("artists with no exact match:")
for artist_row in artist_mentions.loc[ artist_mentions["mentions"] == 0 ].iterrows():
    
    name = artist_row[1]['artist'].lower()
    
    lastname = name.split(" ")[-1]
    print(f'{name} --> lastname: {lastname}')
    lastname_matches = my_utils.search_prompt_splits(exact_matches, lastname)
    if lastname_matches.shape[0] > 0:
        #print(lastname_matches["prompt"].head())
        for index, row in lastname_matches.iterrows():
            prompt = row['prompt'].lower()
            startindex = prompt.index(lastname) -search_padding if prompt.index(lastname) - search_padding > 0 else 0
            end = prompt.index(lastname) + len(lastname) + search_padding if prompt.index(lastname) + len(lastname) + search_padding < len(prompt) else len(prompt)
            print(f'partial match: {prompt[startindex:end]}')

# We can see, that the lastname is not a good enough decision.

            
            

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd

occs = pd.read_parquet('results/artist_mentions.parquet', engine='pyarrow')


x_data = range(occs.shape[0])
y_data = [x[1]["mentions"] for x in occs.iterrows()]
y_data.sort(reverse=True)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(x_data, y_data)
ax.set_xlabel('Rank')
ax.set_ylabel('Occurences')
ax.set_title('Artist Mentions Distribution')

plt.show()


In [ ]:
import my_utils


pd.options.display.max_colwidth = 1000

name = "greg rutkowski"
exact_matches = my_utils.exact_match_dataframe(df, name)

print(f'{name} exact match found {exact_matches.shape[0]} times')
print(f'some prompts:')
print(exact_matches.head(10)["prompt"])

## Top Artist exact matching

In [ ]:
import matplotlib.pyplot as plt

top = 10

artist_mentions.sort_values("mentions", axis=0, ascending=False, inplace=True)

print(f'top {top} artists')
print(artist_mentions.head(top))

fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(1, 1, 1) # fig.add_axes([0,0,1,1])
names = [x for x in artist_mentions["artist"].iloc[0:top]]
values = [x for x in artist_mentions["mentions"].iloc[0:top]]
ax.bar(names,values)
ax.set_title(f'Top {top} mentioned artists')
plt.show()

In [ ]:

import matplotlib.pyplot as plt

mean_num_artists = exact_matches["num_artists"].mean(axis=0)
print(f'on average {mean_num_artists} are mentioned per prompt')

print(f'{exact_matches["num_artists"].sum(axis=0)} total mentions')
  
# labels
labels  = exact_matches["num_artists"].unique()
labels.sort()
  
# values
values = [exact_matches[exact_matches["num_artists"]==x].shape[0] for x in labels]
  
for l, v in zip(labels, values):
    print(f'{v} prompts had {l} artists found')
    
    

plt.rcdefaults()
fig, ax = plt.subplots()

y_pos = range(len(labels))

ax.barh(y_pos, values, align='center')
ax.set_yticks(y_pos, labels=labels)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('amount')
ax.set_title('Num artists distribution')

plt.show()

aggregate_labels = ["zero artists", "one artist", "two or more artists"]
aggregate_values = [exact_matches[exact_matches["num_artists"]==0].shape[0], exact_matches[exact_matches["num_artists"]==1].shape[0], exact_matches[exact_matches["num_artists"]>=2].shape[0]]

print(f'analysis of mention categories')
for l, v in zip(aggregate_labels, aggregate_values):
    print(f'{v} prompts had {l} found')

plt.rcdefaults()
fig, ax = plt.subplots()

y_pos = range(len(aggregate_labels))

ax.barh(y_pos, aggregate_values, align='center')
ax.set_yticks(y_pos, labels=aggregate_labels)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('amount')
ax.set_title('Num artists distribution aggregate')

plt.show()



# Co-occurrences

## For prompts with 2 or more mentioned painters

In [ ]:
import numpy as np


painter_names = artist_mentions.loc[artist_mentions["mentions"] > 0 ]
painter_names = painter_names["artist"].unique()
# painters with more than one mention

# TODO remove this restriction later (or do the analysis only for the top x artists)
painter_names = painter_names[0:3]

print(f'{len(painter_names)} artists with more than 0 mentions: {painter_names}')

cols = ["prompt", "image_name"]

occurences =  pd.DataFrame(columns=cols)
for name in painter_names:
    # create new column for each painter with default value 0
    #occurences[painter_names] = 0
    occurences[name] = pd.Series(dtype=np.float64)
    occurences = occurences.copy()

print("occurences")
print(occurences.columns)

print(f'exact_matches columns {exact_matches.columns}')


# TODO remove this n
#n = 10000
#cp = exact_matches.head(n).copy()
#assert cp.shape[0] == n

cp = exact_matches.copy()


print(cp.head())

print(f'shape before num_artists cut {cp.shape}')
# TODO can we do such a cut??? (scientifically)
# at least 2 artists mentioned for a cooccurence
cp = cp[cp["num_artists"]>=2]
print(f'shape after num_artists cut {cp.shape}')

#print(("extracting finished"))
#print(cp.head())

for _, entry in cp.iterrows():
    #print(entry)
    new_row = dict()
    new_row['image_name'] = entry['image_name']
    new_row['prompt'] = entry['prompt']
    for name in painter_names:
        new_row[name] = 0
    
    # set to one for each painter that really occured
    for name in entry['artists']:
        if name in painter_names:
            new_row[name] = 1
    
    new_row = pd.Series(new_row)
    #print(new_row)
    
    occurences = pd.concat([occurences, new_row.to_frame().T], ignore_index=True).copy()
    # copy to defragment the dataframe

# assert occurences.shape == (n, 2 + len(painter_names)), f'shape is {occurences.shape}'


print("head:")
print(occurences[painter_names].head())

print("covariance:")
# https://www.w3resource.com/pandas/dataframe/dataframe-cov.php
print(occurences[painter_names].cov())

print("finished")

3 artists with more than 0 mentions: ['Alvar Aalto' 'Slim Aarons' 'Edwin Austin Abbey']
occurences
Index(['prompt', 'image_name', 'Alvar Aalto', 'Slim Aarons',
       'Edwin Austin Abbey'],
      dtype='object')
exact_matches columns Index(['prompt', 'image_name', 'artists', 'num_artists'], dtype='object')
                                               prompt  \
0   beautiful porcelain ivory fair face woman biom...   
1   complex 3 d render hyper detailed ultra sharp ...   
15  complex 3 d render hyper detailed ultra sharp ...   
16  complex 3 d render hyper detailed ultra sharp ...   
33  complex 3 d render hyper detailed ultra sharp ...   

                                   image_name            artists  num_artists  
0   3ccdc650-871a-4ad9-9bf2-dc475b83ed32.webp  [iris van herpen]            1  
1   1f1fcb70-63a4-40b1-ada9-2c15fb2ca10a.webp  [iris van herpen]            1  
15  a3154865-5d4c-4ba7-bcb2-7c1b4ed205e5.webp  [iris van herpen]            1  
16  d6e3e37e-d426-41cf-b765-9

## TODO Co-occurrence Network

https://www.kaggle.com/code/itoeiji/simple-co-occurrence-network